This codebook will demonstrate working on the Prophet algorithm using the Adobe historical stock dataset. First, the necessary libraries need to be imported.

In [ ]:
# import necessary libraries
import numpy as np #for powerful array and fast math operations
import pandas as pd #for creating and working on dataframes
import matplotlib.pyplot as plt #for creating plots and charts
import seaborn as sns #another visualization library for better and easier plots
import kagglehub #for interacting with kaggle to download datasets
import os #for interacting with the os to read csv
import prophet #importing the prophet library
from prophet.make_holidays import make_holidays_df #importing holidays for the algorithm

Now the dataset needs to be loaded. It will be loaded and handled with kagglehub and os libraries.

In [ ]:
# Download latest version of the dataset
path = kagglehub.dataset_download("paultimothymooney/stock-market-data")

print("Path to dataset files:", path)

In [ ]:
# `path` points to the downloaded dataset directory
dataset_dir = path

# List files in the dataset directory to identify the data files
print("Files in dataset directory:", os.listdir(dataset_dir))

Now the dataset will be read using the read.csv function and the first few columns along with data types will be observed.

In [ ]:
# Load Adobe stock data
adbe_path = os.path.join(dataset_dir, "stock_market_data", "nasdaq", "csv", "ADBE.csv")
adbe_df = pd.read_csv(adbe_path)

# Inspect first rows
print(adbe_df.head())
print(adbe_df.info())

The output from the last cell showed that the 'Date' column contains object datatype. But Prophet requires the the date to be datetime format. The following cell will convert the dates to datetime format.

In [ ]:
# Turning dates into datetime format
adbe_df['Date'] = pd.to_datetime(adbe_df['Date'], dayfirst = True)

In [ ]:
adbe_df.tail(n=20)

Now the dataset needs to be checked for any missing dates. For this, I'll first find out the total range of date the data spans over. From this, I'll calculate the total calendar days. Then I'll subtract from it the available dates in my dataset, the holidays, and the weekends. This will give me the missing dates.

In [ ]:
# Find the total number of dates available in the dataset
start_date = adbe_df['Date'].min()
end_date = adbe_df['Date'].max()

print(f"Date range: {start_date} to {end_date}")

In [ ]:
# Calculate the total number of calendar days in the timeline
full_timeline = pd.date_range(start=start_date, end=end_date, freq='D')
total_calendar_days = len(full_timeline)
# Calculate the number of days available in the dataset
available_data_days = len(adbe_df)
print(f"Total calendar days in the timeline: {total_calendar_days}")
print(f"Number of days with available data: {available_data_days}")

In [ ]:
# Calculate the number of weekends in the timeline
number_of_weekends = sum(full_timeline.dayofweek >= 5)
print(f"Number of weekend days (Sat/Sun): {number_of_weekends}")

In [ ]:
# Calculate the number of holidays for the period
year_list = range(start_date.year, end_date.year + 1)
holidays_df = make_holidays_df(year_list=year_list, country='US')
# Filter holidays to be within the timeline and not on a weekend
holidays_in_timeline = holidays_df[(holidays_df['ds'] >= start_date) & (holidays_df['ds'] <= end_date) & (holidays_df['ds'].dt.dayofweek < 5)]
number_of_holidays = len(holidays_in_timeline)
print(f"Number of public holiday weekdays: {number_of_holidays}")

In [ ]:
# find the days that are missing on the dataset
missing_days = total_calendar_days - available_data_days - number_of_weekends - number_of_holidays
print(f"Missing days in the dataset: {missing_days}")

Here the negative number is because prophet's holiday library includes some holidays on which stock market stays open. Since the number is only 45 in the span of 36 years, this is minimal and therefore it is ignoreable.

Now I'll prepare my data for Prophet algorithm. Since 2020 contains the COVID pandemic, an anomaly that caused a lot of volatility, I'll train and test my model with data up till 2020.

In [ ]:
# Filter a range for model training
start_date = '1986-08-13'
end_date = '2019-12-31'

# Filter ADBE stock
adbe_train = adbe_df[(adbe_df['Date'] >= start_date) & (adbe_df['Date'] <= end_date)].copy()

adbe_train = adbe_train.set_index('Date')

In [ ]:
# inspect the dataset
adbe_train.head(n=20)

Now I'll check for any duplicates dates, because each date should only have one value for each of the columns and therefore only one row.

In [ ]:
# check for duplictaes
duplicate = adbe_train.reset_index().duplicated(subset=['Date'])
adbe_train[duplicate.values]

Since there are no duplicates found, I'll go forward to check the relationship between the variables. The 'pairplot' function shows the realationship between each variable in separate graphs, making it easy to inspect the realtion and find an anomaly.

In [ ]:
sns.pairplot(adbe_train)
plt.show()

Now I'll create a new dataframe with only the necessary columns. Prophet requires the 'Dates' column to be named 'ds', and the column we'll predict (here 'Adjusted Close') to be named 'y'. Tlog transform the variables, because it stabilizes the variables and handles non-linear growth, that are usually seen in stock data.

In [ ]:
train_df = adbe_train[['Adjusted Close']].reset_index()
train_df.rename(columns={'Date': 'ds', 'Adjusted Close': 'y'}, inplace=True)
train_df.head(n=20)

In [ ]:
# log transformation
import numpy as np
train_df_log = train_df.copy()
train_df_log['y'] = np.log1p(train_df_log['y'])
train_df_log.head(n=10)

I'll train 2 models. At first will be using the built in methods without any added regressors. Then I'll add some regressors to see if that helps the model's predictions.

I'll use the same train size, horizon, and storage for metrices for comparability. i'll train on 90% of data using a 60 day horizon for forecasting.

In [ ]:
# Parameters for model traininng
initial_train_size = int(0.9 * len(train_df_log))
horizon = 60
n_iterations = (len(train_df_log) - initial_train_size) // horizon

# Storage for metrics
metrics_1 = {'mae': [], 'rmse': [], 'mape': [], 'r2': []}

Now I'll import holidays from the Prophet's built in methods. Adding holidays is importatnt because holidays introduce sudden spikes in stock proces.

In [ ]:
from prophet import Prophet
from prophet.make_holidays import make_holidays_df

# Make holidays DataFrame for US
holidays = make_holidays_df(year_list=range(1986, 2022), country='US')

Now I'll train the first model. It'll be a basic model without regressors using the Prophet's built in cross validation function for this.

In [ ]:
# built in cross validation model (without any added regressors)
import prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import logging

# Suppress warnings using logging
logging.getLogger('prophet').setLevel(logging.ERROR)

# Initialize and tune Prophet with built-in components
model_1 = Prophet(
    growth='linear', #assuming data grows and shrinks at steady rate without caps
    daily_seasonality=False, #since there's no hourly data
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='additive', #because the data is log-transformed
    changepoint_prior_scale=0.5, #higher value for more flexibility and trend changes
    interval_width=0.95, #95% confidence interval
    uncertainty_samples=1000, #more samples for more stable interval
    seasonality_prior_scale=0.1, #small to smooth out the seasonality to reduce overfitting
    holidays=holidays
)

# Add quarterly seasonality to reflect quaterly patterns
model_1.add_seasonality(name='quarterly', period=91.3, fourier_order=5) #stock data usually show quarterly seasonality

print(f"\nTraining Prophet on {len(train_df_log)} data points")
model_1.fit(train_df_log)

# Cross-validation with parameter tuning
print(f"\nRunning cross-validation:")
print(f"  Initial training size: {initial_train_size} days (90% of data)")
print(f"  Horizon: {horizon} days")
print(f"  Number of folds: {n_iterations}")

df_cv = cross_validation(
    model_1,
    initial=f'{initial_train_size} days',
    horizon=f'{horizon} days',
    period=f'{horizon} days',
    parallel='processes'
)

print(f"  Total predictions made: {len(df_cv)}")

# Reverse log transformation for actual scale metrics
df_cv['y_actual'] = np.expm1(df_cv['y'])
df_cv['yhat_actual'] = np.expm1(df_cv['yhat'])
df_cv['yhat_lower_actual'] = np.expm1(df_cv['yhat_lower'])
df_cv['yhat_upper_actual'] = np.expm1(df_cv['yhat_upper'])

# Store predictions dataframe
predictions_cv_no_regressors = df_cv.copy()

# Calculate metrics in original scale
mae = mean_absolute_error(df_cv['y_actual'], df_cv['yhat_actual'])
rmse = np.sqrt(mean_squared_error(df_cv['y_actual'], df_cv['yhat_actual']))
mape = np.mean(np.abs((df_cv['y_actual'] - df_cv['yhat_actual']) / (df_cv['y_actual'] + 1e-8))) * 100
r2 = r2_score(df_cv['y_actual'], df_cv['yhat_actual'])

# Coverage analysis
in_interval = np.sum((df_cv['y_actual'] >= df_cv['yhat_lower_actual']) &
                     (df_cv['y_actual'] <= df_cv['yhat_upper_actual']))
coverage_pct = (in_interval / len(df_cv)) * 100
avg_width_pct = np.mean((df_cv['yhat_upper_actual'] - df_cv['yhat_lower_actual']) /
                        df_cv['y_actual']) * 100

print("Model results with built_in components:")

print(f"\nMAE      = {mae:.3f}")
print(f"RMSE     = {rmse:.3f}")
print(f"MAPE     = {mape:.3f}")
print(f"R²       = {r2:.4f}")

print(f"\nCOVERAGE = {coverage_pct:.1f}%")
print(f"AVG WIDTH = {avg_width_pct:.1f}%")

Although this model resulted in decent r2 value and coverage, the MAE, RMSE, and MAPE values are not ideal for a stock forecasting model.

I'll use the following cell to save the predictions now, so I don't have to run it everytime my runtime disconnects to get the results.

In [ ]:
# Save the predictions after running model 1
import pickle
predictions_cv_no_regressors.to_csv('predictions_cv_no_regressors.csv', index=False)
print("Saved predictions_cv_no_regressors")

In [ ]:
# load the predictions to chcek
predictions_cv_no_regressors = pd.read_csv('predictions_no_regressors.csv', parse_dates=['ds'])
predictions_cv_no_regressors.head()

Now I'll visualize all the components (weekly, yearly, and quately seasonality,holiday, and trend) using Prophet's built in plot_components. This gives us a clear idea about the patterns.

In [ ]:
# Visualize all the components of the dataset
forecast_model_1 = model_1.predict(train_df_log)

figure_1 = model_1.plot_components(forecast_model_1)
plt.show()

Now I'll visualize the MAPE metric. It shows 4% instead of 16% because it's calculated in the log transformed space.

In [ ]:
from prophet.plot import plot_cross_validation_metric
# You can change 'mape' to 'rmse', 'mae', or 'mdape'.
fiure_2 = plot_cross_validation_metric(df_cv, metric='mape')
plt.title("Cross-Validation: MAPE vs. Horizon")
plt.show()

Now I'll train the model creating and adding some regressors to see if they improve the model's prediction. The regressors are not exogenous in nature, and are derived from this dataset. I'll use autoregression to predict future regressors and avoid any data leakage. The train size and horizon will be the same as before.

In [ ]:
# Parameters for iterative traininng
initial_train_size = int(0.9 * len(train_df_log))
horizon = 60
n_iterations = (len(train_df_log) - initial_train_size) // horizon

In [ ]:
# Storage for metrics
metrics = {'mae': [], 'rmse': [], 'mape': [], 'r2': []}

The following model includes the following regressors: lag 1, 2, 3, and 7 (previous days' closing prices), simple moving average 7, and 30 (using rolling window means), 5 and 10 days' momentum, volatility 7 and 30 (using rolling window standard deviations), and trend strength (deviation from sma 30).

In [ ]:
# Walk forward validation and performance metrics with autoregression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Main Walk-Forward Loop
train_end = initial_train_size
all_fold_forecasts = []
coverage_metrics = {'in_coverage': [], 'coverage_width': []}

# Store all predictions for later visualization
all_predictions_list = []

for i in range(n_iterations):
    # Define current training and testing windows
    current_train = train_df_log.iloc[:train_end].copy()
    test_start = train_end
    test_end = min(train_end + horizon, len(train_df_log))
    test_df = train_df_log.iloc[test_start:test_end].copy()

    if len(test_df) == 0:
        break

    print(f"Iteration {i+1}/{n_iterations}: Training on {len(current_train)} data points, testing on {len(test_df)}.")

    # Recalculate features on the loop to avoid leakage
    current_train_clean = current_train[['ds', 'y']].copy()

    # Include basic lags
    current_train_clean['lag1'] = current_train_clean['y'].shift(1)
    current_train_clean['lag2'] = current_train_clean['y'].shift(2)
    current_train_clean['lag3'] = current_train_clean['y'].shift(3)
    current_train_clean['lag7'] = current_train_clean['y'].shift(7)

    # Include moving averages
    current_train_clean['sma_7'] = current_train_clean['y'].rolling(window=7, min_periods=1).mean()
    current_train_clean['sma_30'] = current_train_clean['y'].rolling(window=30, min_periods=1).mean()

    # Include momentum indicators
    current_train_clean['momentum_5'] = current_train_clean['y'] - current_train_clean['y'].shift(5)
    current_train_clean['momentum_10'] = current_train_clean['y'] - current_train_clean['y'].shift(10)

    # Include volatility (rolling std)
    current_train_clean['volatility_7'] = current_train_clean['y'].rolling(window=7, min_periods=1).std()
    current_train_clean['volatility_30'] = current_train_clean['y'].rolling(window=30, min_periods=1).std()

    # Include trend strength (price relative to moving average)
    current_train_clean['trend_strength'] = current_train_clean['y'] - current_train_clean['sma_30']

    # Drop rows with NaN
    current_train_clean = current_train_clean.dropna()

    if len(current_train_clean) < 50:
        print("  Insufficient training data after feature engineering. Skipping.")
        continue

    # Initialize and Fit Model with Reduced Seasonality
    model_2 = Prophet(
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True,
        seasonality_mode='additive',
        changepoint_prior_scale=0.05,  # Lower value is less flexible (prevents overfitting)
        interval_width=0.95,
        uncertainty_samples=1000,
        seasonality_prior_scale=0.1,
        holidays=holidays
    )

    # Add regressors to the model
    regressors = ['lag1', 'lag2', 'lag3', 'lag7', 'sma_7', 'sma_30', 'momentum_5', 'momentum_10', 'volatility_7', 'volatility_30', 'trend_strength']

    for reg in regressors:
        model_2.add_regressor(reg)
    model_2.add_seasonality(name='quarterly', period=91.3, fourier_order=5)

    model_2.fit(current_train_clean)

    # Forecasting with Actual Values (not recursive)
    history_y = current_train_clean['y'].values.tolist()
    fold_predictions = []
    actual_dates = test_df['ds'].values

    for step in range(len(test_df)):
        # Create future dataframe
        future_step_df = pd.DataFrame({'ds': [actual_dates[step]]})

        # Calculate regressors from history
        history_array = np.array(history_y)

        future_step_df['lag1'] = history_array[-1]
        future_step_df['lag2'] = history_array[-2]
        future_step_df['lag3'] = history_array[-3]
        future_step_df['lag7'] = history_array[-7] if len(history_array) >= 7 else history_array[0]

        future_step_df['sma_7'] = history_array[-7:].mean() if len(history_array) >= 7 else history_array.mean()
        future_step_df['sma_30'] = history_array[-30:].mean() if len(history_array) >= 30 else history_array.mean()

        future_step_df['momentum_5'] = history_array[-1] - (history_array[-6] if len(history_array) >= 6 else history_array[0])
        future_step_df['momentum_10'] = history_array[-1] - (history_array[-11] if len(history_array) >= 11 else history_array[0])

        future_step_df['volatility_7'] = np.std(history_array[-7:]) if len(history_array) >= 7 else 0
        future_step_df['volatility_30'] = np.std(history_array[-30:]) if len(history_array) >= 30 else 0

        sma_30_current = history_array[-30:].mean() if len(history_array) >= 30 else history_array.mean()
        future_step_df['trend_strength'] = history_array[-1] - sma_30_current

        # Make prediction with intervals
        forecast_step = model_2.predict(future_step_df)
        fold_predictions.append(forecast_step)

        # Use actual value for next iteration
        actual_log_value = test_df.iloc[step]['y']
        history_y.append(actual_log_value)

    # Calculate Metrics with Coverage
    fold_forecast_df = pd.concat(fold_predictions).reset_index(drop=True)
    all_fold_forecasts.append(fold_forecast_df)

    # Reverse log transformation
    y_true_original_2 = np.expm1(test_df['y'].values)
    y_pred_original_2 = np.expm1(fold_forecast_df['yhat'].values)
    y_lower_original_2 = np.expm1(fold_forecast_df['yhat_lower'].values)
    y_upper_original_2 = np.expm1(fold_forecast_df['yhat_upper'].values)

    # Store predictions for this fold
    for j in range(len(test_df)):
        all_predictions_list.append({
            'ds': test_df.iloc[j]['ds'],
            'y_actual_log': test_df.iloc[j]['y'],
            'y_actual': y_true_original_2[j],
            'y_pred_log': fold_forecast_df.iloc[j]['yhat'],
            'y_pred': y_pred_original_2[j],
            'y_lower_log': fold_forecast_df.iloc[j]['yhat_lower'],
            'y_lower': y_lower_original_2[j],
            'y_upper_log': fold_forecast_df.iloc[j]['yhat_upper'],
            'y_upper': y_upper_original_2[j],
            'fold': i + 1
        })

    # Standard metrics
    mae = mean_absolute_error(y_true_original_2, y_pred_original_2)
    rmse = np.sqrt(mean_squared_error(y_true_original_2, y_pred_original_2))
    mape = np.mean(np.abs((y_true_original_2 - y_pred_original_2) / (y_true_original_2 + 1e-8))) * 100
    r2 = r2_score(y_true_original_2, y_pred_original_2)

    # Coverage metrics
    in_interval = np.sum((y_true_original_2 >= y_lower_original_2) & (y_true_original_2 <= y_upper_original_2))
    coverage = (in_interval / len(y_true_original_2)) * 100
    avg_interval_width = np.mean(y_upper_original_2 - y_lower_original_2)
    avg_interval_width_pct = np.mean((y_upper_original_2 - y_lower_original_2) / y_true_original_2) * 100


    metrics['mae'].append(mae)
    metrics['rmse'].append(rmse)
    metrics['mape'].append(mape)
    metrics['r2'].append(r2)
    coverage_metrics['in_coverage'].append(coverage)
    coverage_metrics['coverage_width'].append(avg_interval_width_pct)

    # Move the training window (candlestick method)
    train_end += horizon

    print(f"    MAE: {mae:.2f} | RMSE: {rmse:.2f} | MAPE: {mape:.2f}% | R²: {r2:.3f}")
    print(f"    Coverage: {coverage:.1f}% | Avg Width: {avg_interval_width_pct:.1f}%")

# Create comprehensive predictions dataframe
predictions_df_with_regressors_nr = pd.DataFrame(all_predictions_list)

# Aggregate and Display Final Metrics
print("AGGREGATED METRICS (Walk-Forward Validation with added regressors)")

agg_metrics = {k: np.mean(v) for k, v in metrics.items()}
for k, v in agg_metrics.items():
    std_val = np.std(metrics[k])
    print(f"{k.upper():8} = {v:7.3f} (±{std_val:.3f})")

print(f"\nCOVERAGE  = {np.mean(coverage_metrics['in_coverage']):7.1f}% (±{np.std(coverage_metrics['in_coverage']):.1f}%)")
print(f"AVG WIDTH = {np.mean(coverage_metrics['coverage_width']):7.1f}% (±{np.std(coverage_metrics['coverage_width']):.1f}%)")


Although this model produces decent results, this is not truly recursive in nature. Rather after each predction, it uses the original values of regressors, assuming each day the values up to the previous day will be fed to the model.

Now similar to the model before, I'll store and check the predictions of this model.

In [ ]:
# Saving after running model with regressors
import pickle
predictions_df_with_regressors_nr.to_csv('predictions_with_regressors_nr.csv', index=False)
with open('metrics_with_reg.pkl', 'wb') as f:
    pickle.dump(metrics, f)
print("Saved predictions_df_with_regressors_nr")

In [ ]:
predictions_df_with_regressors_nr = pd.read_csv('predictions_with_regressors_nr.csv', parse_dates=['ds'])
predictions_df_with_regressors_nr.head()

Now I'll train a true recursive model. Since the errors can compound on such models, I'll use a shorter horizon of 14 days and a higher changepoint prior scale (0.5).

In [ ]:
# Parameters for iterative traininng
initial_train_size = int(0.9 * len(train_df_log))
horizon = 14
n_iterations = (len(train_df_log) - initial_train_size) // horizon
# Storage for metrics
metrics = {'mae': [], 'rmse': [], 'mape': [], 'r2': []}

In [ ]:
# Walk forward validation and performance metrics with autoregression (True Recursive)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Main Walk-Forward Loop
train_end = initial_train_size
all_fold_forecasts = []
coverage_metrics = {'in_coverage': [], 'coverage_width': []}

# Store all predictions for later visualization
all_predictions_list = []

for i in range(n_iterations):
    # Define current training and testing windows
    current_train = train_df_log.iloc[:train_end].copy()
    test_start = train_end
    test_end = min(train_end + horizon, len(train_df_log))
    test_df = train_df_log.iloc[test_start:test_end].copy()

    if len(test_df) == 0:
        break

    print(f"Iteration {i+1}/{n_iterations}: Training on {len(current_train)} data points, testing on {len(test_df)}.")

    # Recalculate features on the loop to avoid leakage
    current_train_clean = current_train[['ds', 'y']].copy()

    # Include basic lags
    current_train_clean['lag1'] = current_train_clean['y'].shift(1)
    current_train_clean['lag2'] = current_train_clean['y'].shift(2)
    current_train_clean['lag3'] = current_train_clean['y'].shift(3)
    current_train_clean['lag7'] = current_train_clean['y'].shift(7)

    # Include moving averages
    current_train_clean['sma_7'] = current_train_clean['y'].rolling(window=7, min_periods=1).mean()
    current_train_clean['sma_30'] = current_train_clean['y'].rolling(window=30, min_periods=1).mean()

    # Include momentum indicators
    current_train_clean['momentum_5'] = current_train_clean['y'] - current_train_clean['y'].shift(5)
    current_train_clean['momentum_10'] = current_train_clean['y'] - current_train_clean['y'].shift(10)

    # Include volatility (rolling std)
    current_train_clean['volatility_7'] = current_train_clean['y'].rolling(window=7, min_periods=1).std()
    current_train_clean['volatility_30'] = current_train_clean['y'].rolling(window=30, min_periods=1).std()

    # Include trend strength (price relative to moving average)
    current_train_clean['trend_strength'] = current_train_clean['y'] - current_train_clean['sma_30']

    # Drop rows with NaN
    current_train_clean = current_train_clean.dropna()

    if len(current_train_clean) < 50:
        print("  Insufficient training data after feature engineering. Skipping.")
        continue

    # Initialize and Fit Model with Reduced Seasonality
    model_2 = Prophet(
        daily_seasonality=False,
        weekly_seasonality=True,
        yearly_seasonality=True,
        seasonality_mode='additive',
        changepoint_prior_scale=0.5,  # Lower value is less flexible (prevents overfitting)
        interval_width=0.95,
        uncertainty_samples=1000,
        seasonality_prior_scale=0.1,
        holidays=holidays
    )

    # Add regressors to the model
    regressors = ['lag1', 'lag2', 'lag3', 'lag7', 'sma_7', 'sma_30', 'momentum_5', 'momentum_10', 'volatility_7', 'volatility_30', 'trend_strength']

    for reg in regressors:
        model_2.add_regressor(reg)
    model_2.add_seasonality(name='quarterly', period=91.3, fourier_order=5)

    model_2.fit(current_train_clean)

    # TRUE RECURSIVE Forecasting (using predicted values)
    history_y = current_train_clean['y'].values.tolist()
    fold_predictions = []
    actual_dates = test_df['ds'].values

    for step in range(len(test_df)):
        # Create future dataframe
        future_step_df = pd.DataFrame({'ds': [actual_dates[step]]})

        # Calculate regressors from history
        history_array = np.array(history_y)

        future_step_df['lag1'] = history_array[-1]
        future_step_df['lag2'] = history_array[-2]
        future_step_df['lag3'] = history_array[-3]
        future_step_df['lag7'] = history_array[-7] if len(history_array) >= 7 else history_array[0]

        future_step_df['sma_7'] = history_array[-7:].mean() if len(history_array) >= 7 else history_array.mean()
        future_step_df['sma_30'] = history_array[-30:].mean() if len(history_array) >= 30 else history_array.mean()

        future_step_df['momentum_5'] = history_array[-1] - (history_array[-6] if len(history_array) >= 6 else history_array[0])
        future_step_df['momentum_10'] = history_array[-1] - (history_array[-11] if len(history_array) >= 11 else history_array[0])

        future_step_df['volatility_7'] = np.std(history_array[-7:]) if len(history_array) >= 7 else 0
        future_step_df['volatility_30'] = np.std(history_array[-30:]) if len(history_array) >= 30 else 0

        sma_30_current = history_array[-30:].mean() if len(history_array) >= 30 else history_array.mean()
        future_step_df['trend_strength'] = history_array[-1] - sma_30_current

        # Make prediction with intervals
        forecast_step = model_2.predict(future_step_df)
        fold_predictions.append(forecast_step)

        # TRUE RECURSIVE: Use PREDICTED value for next iteration
        predicted_log_value = forecast_step['yhat'].values[0]
        history_y.append(predicted_log_value)

    # Calculate Metrics with Coverage
    fold_forecast_df = pd.concat(fold_predictions).reset_index(drop=True)
    all_fold_forecasts.append(fold_forecast_df)

    # Reverse log transformation
    y_true_original_2 = np.expm1(test_df['y'].values)
    y_pred_original_2 = np.expm1(fold_forecast_df['yhat'].values)
    y_lower_original_2 = np.expm1(fold_forecast_df['yhat_lower'].values)
    y_upper_original_2 = np.expm1(fold_forecast_df['yhat_upper'].values)

    # Store predictions for this fold
    for j in range(len(test_df)):
        all_predictions_list.append({
            'ds': test_df.iloc[j]['ds'],
            'y_actual_log': test_df.iloc[j]['y'],
            'y_actual': y_true_original_2[j],
            'y_pred_log': fold_forecast_df.iloc[j]['yhat'],
            'y_pred': y_pred_original_2[j],
            'y_lower_log': fold_forecast_df.iloc[j]['yhat_lower'],
            'y_lower': y_lower_original_2[j],
            'y_upper_log': fold_forecast_df.iloc[j]['yhat_upper'],
            'y_upper': y_upper_original_2[j],
            'fold': i + 1
        })

    # Standard metrics
    mae = mean_absolute_error(y_true_original_2, y_pred_original_2)
    rmse = np.sqrt(mean_squared_error(y_true_original_2, y_pred_original_2))
    mape = np.mean(np.abs((y_true_original_2 - y_pred_original_2) / (y_true_original_2 + 1e-8))) * 100
    r2 = r2_score(y_true_original_2, y_pred_original_2)

    # Coverage metrics
    in_interval = np.sum((y_true_original_2 >= y_lower_original_2) & (y_true_original_2 <= y_upper_original_2))
    coverage = (in_interval / len(y_true_original_2)) * 100
    avg_interval_width = np.mean(y_upper_original_2 - y_lower_original_2)
    avg_interval_width_pct = np.mean((y_upper_original_2 - y_lower_original_2) / y_true_original_2) * 100

    metrics['mae'].append(mae)
    metrics['rmse'].append(rmse)
    metrics['mape'].append(mape)
    metrics['r2'].append(r2)
    coverage_metrics['in_coverage'].append(coverage)
    coverage_metrics['coverage_width'].append(avg_interval_width_pct)

    # Move the training window (candlestick method)
    train_end += horizon

    print(f"    MAE: {mae:.2f} | RMSE: {rmse:.2f} | MAPE: {mape:.2f}% | R²: {r2:.3f}")
    print(f"    Coverage: {coverage:.1f}% | Avg Width: {avg_interval_width_pct:.1f}%")

# Create comprehensive predictions dataframe
predictions_df_with_regressors = pd.DataFrame(all_predictions_list)

# Aggregate and Display Final Metrics
print("AGGREGATED METRICS (Walk-Forward Validation with TRUE RECURSIVE)")

agg_metrics = {k: np.mean(v) for k, v in metrics.items()}
for k, v in agg_metrics.items():
    std_val = np.std(metrics[k])
    print(f"{k.upper():8} = {v:7.3f} (±{std_val:.3f})")

print(f"\nCOVERAGE  = {np.mean(coverage_metrics['in_coverage']):7.1f}% (±{np.std(coverage_metrics['in_coverage']):.1f}%)")
print(f"AVG WIDTH = {np.mean(coverage_metrics['coverage_width']):7.1f}% (±{np.std(coverage_metrics['coverage_width']):.1f}%)")

Although this has improved MAE, RMSE, and MAPE, the r2 and coverage values have worsened. Now I'll save these results as well.

In [ ]:
# Save predictions after running model with regressors
import pickle
predictions_df_with_regressors.to_csv('predictions_with_regressors.csv', index=False)
with open('metrics_with_reg.pkl', 'wb') as f:
    pickle.dump(metrics, f)
print("Saved predictions_df_with_regressors")

In [ ]:
# load predictions to check
predictions_df_with_regressors = pd.read_csv('predictions_with_regressors.csv', parse_dates=['ds'])
predictions_df_with_regressors.head()

Now I'll compare all my model's metrices with a persistence model. I'll use the same training size and horizon (60 days) as my previous models to maintain comparability.

In [ ]:
# Testing model's validation with persistance model (walk-forward)
train_end = initial_train_size
persistence_metrics = {'mae': [], 'rmse': [], 'mape': [], 'r2': []}

# Store all predictions
all_predictions_list_persistence = []

for i in range(n_iterations):
    # Define train and test windows
    test_start = train_end
    test_end = min(train_end + horizon, len(train_df_log))
    test_df = train_df_log.iloc[test_start:test_end].copy()

    if len(test_df) == 0:
        break

    # Intoducing persistence prediction: tomorrow's price = today's price
    y_pred_log = test_df['y'].shift(1).fillna(test_df['y'].iloc[0])

    # Transform back to original scale
    y_true_pers = np.expm1(test_df['y'].values)
    y_pred_pers = np.expm1(y_pred_log.values)

    # Store predictions for this fold
    for j in range(len(test_df)):
        all_predictions_list_persistence.append({
            'ds': test_df.iloc[j]['ds'],
            'y_actual_log': test_df.iloc[j]['y'],
            'y_actual': y_true_pers[j],
            'y_pred_log': y_pred_log.iloc[j],
            'y_pred': y_pred_pers[j],
            'fold': i + 1
        })

    # Calculate metrics
    mae = mean_absolute_error(y_true_pers, y_pred_pers)
    rmse = np.sqrt(mean_squared_error(y_true_pers, y_pred_pers))
    mape = np.mean(np.abs((y_true_pers - y_pred_pers) / (y_true_pers + 1e-8))) * 100
    r2 = r2_score(y_true_pers, y_pred_pers)

    # Store metrics
    persistence_metrics['mae'].append(mae)
    persistence_metrics['rmse'].append(rmse)
    persistence_metrics['mape'].append(mape)
    persistence_metrics['r2'].append(r2)

    # Move window forward (candlestick method)
    train_end += horizon

# Create predictions dataframe
predictions_df_persistence = pd.DataFrame(all_predictions_list_persistence)

# Aggregated Results
print(f"\nEvaluated on {n_iterations} folds with {horizon}-day horizon\n")

print("PERSISTENCE MODEL RESULTS")

for metric_name, values in persistence_metrics.items():
    mean_val = np.mean(values)
    std_val = np.std(values)
    print(f"{metric_name.upper():8} = {mean_val:7.3f} (±{std_val:.3f})")

Now I'll save the results of the persistence model as well for future comparison. The model results in slightly better but comparable metrics compared to the model with regressors (Model 2 with autoregression, which is not true recursive).

In [ ]:
# Save predictions after persistence model
predictions_df_persistence.to_csv('predictions_persistence.csv', index=False)
print("Saved predictions_df_persistence")

In [ ]:
# load predictions to check values
predictions_df_persistence = pd.read_csv('predictions_persistence.csv', parse_dates=['ds'])
predictions_df_persistence.head()

Now I'll create a table with side by side precitions of 4 models along with the actual close value.

In [ ]:
# Prepare model 1 (cross validation) predictions
cv_predictions = predictions_cv_no_regressors[['ds', 'y_actual', 'yhat_actual']].copy()
cv_predictions = cv_predictions.rename(columns={'yhat_actual': 'y_pred_no_reg'})

# Start with the model without regressors
comparison_all_models = cv_predictions.copy()

# Merge with the model with regressors
comparison_all_models = comparison_all_models.merge(predictions_df_with_regressors_nr[['ds', 'y_pred']], on='ds', how='left')
comparison_all_models = comparison_all_models.rename(columns={'y_pred': 'y_pred_with_reg_nr'})

# Merge with the true recursive model with regressors
comparison_all_models = comparison_all_models.merge(predictions_df_with_regressors[['ds', 'y_pred']], on='ds', how='left')
comparison_all_models = comparison_all_models.rename(columns={'y_pred': 'y_pred_with_reg'})

# Merge with persistence model
comparison_all_models = comparison_all_models.merge(predictions_df_persistence[['ds', 'y_pred']], on='ds', how='left')
comparison_all_models = comparison_all_models.rename(columns={'y_pred': 'y_pred_persistence'})

# Reorder columns
comparison_all_models = comparison_all_models[['ds', 'y_actual', 'y_pred_no_reg',
                                                 'y_pred_with_reg_nr', 'y_pred_with_reg', 'y_pred_persistence']]

# Count missing values
print("MISSING VALUES CHECK")
print(f"No Regressors (CV):      {comparison_all_models['y_pred_no_reg'].isna().sum()} missing")
print(f"With Regressors (NR):    {comparison_all_models['y_pred_with_reg_nr'].isna().sum()} missing")
print(f"With Regressors (True Recursive):   {comparison_all_models['y_pred_with_reg'].isna().sum()} missing")
print(f"Persistence (WFV):       {comparison_all_models['y_pred_persistence'].isna().sum()} missing")

# Keep only rows where ALL three models have predictions
comparison_all_models_complete = comparison_all_models.dropna(subset=['y_pred_no_reg', 'y_pred_with_reg_nr', 'y_pred_with_reg', 'y_pred_persistence'])

comparison_all_models_complete.head()

Now I'll visualize the predictions of all 4 models to see how each model is actually performing. I'll plot the predictions of all these models besides the original y value (Adjusted Close Price).

In [ ]:
# visualize all 3 models' predictions side by side
plt.figure(figsize=(20, 10))
import matplotlib.pyplot as plt
import seaborn as sns
plt.plot(comparison_all_models_complete['ds'], comparison_all_models_complete['y_actual'], label='Actual', color = 'yellow', alpha = 0.6)
plt.plot(comparison_all_models_complete['ds'], comparison_all_models_complete['y_pred_no_reg'], label='Model 1 (without regressors)', color = 'orange', alpha=0.6)
plt.plot(comparison_all_models_complete['ds'], comparison_all_models_complete['y_pred_with_reg_nr'], label='Model 2 (with regressors)', color = 'red', alpha=0.6)
plt.plot(comparison_all_models_complete['ds'], comparison_all_models_complete['y_pred_with_reg'], label='Model 2 (true recursive)', color = 'blue', alpha=0.6)
plt.plot(comparison_all_models_complete['ds'], comparison_all_models_complete['y_pred_persistence'], label='Persistence', color = 'green', alpha=0.6)
plt.xticks(rotation=45) # Rotate x-axis labels for better readability
plt.xlabel('Date')
plt.ylabel('Closing Price')
plt.title('Comparison of 3 Models')
plt.legend()
plt.show()

Here is the direct link to the dataset. The dataset is Adobe stock data from the datacard at this address https://www.kaggle.com/datasets/paultimothymooney/stock-market-data/data.